In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import os
import scipy.stats
import seaborn as sns
import matplotlib.pyplot as plt

INPUT_PATH = Path('../input')
OUTPUT_PATH = Path('../output')
pd.set_option('display.float_format', str)

In [2]:
%%time

transactions = pd.read_csv(INPUT_PATH / 'trxn.csv')
assets_under_management = pd.read_csv(INPUT_PATH / 'aum.csv')
balance = pd.read_csv(INPUT_PATH / 'balance.csv')
client = pd.read_csv(INPUT_PATH / 'client.csv')
campaigns = pd.read_csv(INPUT_PATH / 'com.csv')
deals = pd.read_csv(INPUT_PATH / 'deals.csv')
dict_merchant_category_code = pd.read_csv(INPUT_PATH / 'dict_mcc.csv')
payments = pd.read_csv(INPUT_PATH / 'payments.csv')
funnel = pd.read_csv(INPUT_PATH / 'funnel.csv')
appl = pd.read_csv(INPUT_PATH / 'appl.csv')

/opt/conda/lib/python3.7/site-packages/decorator.py:231: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  return caller(func, *(extras + args), **kw)


CPU times: user 7.06 s, sys: 498 ms, total: 7.55 s
Wall time: 7.55 s


In [3]:
data = funnel.copy()

In [4]:
def get_feature_total(df, col_name):
    return data['client_id'].map(df.groupby(['client_id', col_name]).size().index.get_level_values('client_id').value_counts()).fillna(0)


def get_feature_most_common(df, col_name, fill_na_value):
    return data['client_id'].map(df.groupby('client_id')[col_name].agg(lambda x: scipy.stats.mode(x)[0][0])).fillna(fill_na_value)

# Funnel

In [5]:
funnel

,client_id,sale_flg,sale_amount,contacts,feature_1,client_segment,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,region_cd
0,7513301859607023584,0,NaN,1,7,13.0,571533.0,15717.0,0.0,0.0,0.0,571852.0,472605.0,10.4,12548.0,86.0
1,9157009756404187626,0,NaN,1,3,13.0,3642369.0,94787.0,0.0,0.0,84823.0,3642369.0,3314257.0,8.9,77210.0,2.0
2,-1893104556496814867,0,NaN,1,5,16.0,352826.0,5500.0,0.0,6822.0,0.0,265893.0,204534.0,8.9,5508.0,52.0
3,6886062013213911831,0,NaN,1,4,3.0,6070615.0,40580.0,0.0,30401.0,0.0,2005731.0,1825051.0,7.9,40583.0,86.0
4,-8156468515495593794,1,138018.05000000002,1,7,14.0,3642369.0,97156.0,81488.0,0.0,160308.0,3642369.0,3314257.0,10.4,78108.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21493,4662551505651924284,0,NaN,2,3,3.0,6070615.0,97616.0,21921.0,85635.0,89802.0,4826139.0,4391391.0,7.9,97626.0,86.0
21494,-8968737688687691353,0,NaN,1,9,14.0,1821184.0,2428.0,3952.0,0.0,13938.0,92273.0,71641.0,19.9,2440.0,78.0
21495,-5799097497074119478,0,NaN,1,2,16.0,6070615.0,64991.0,0.0,13450.0,0.0,3212569.0,2923175.0,7.9,64998.0,69.0
21496,-2264802671063321355,1,43882.85000000001,2,8,4.0,2282408.0,5832.0,16185.0,0.0,82182.0,230683.0,177449.0,17.9,5845.0,59.0


In [6]:
funnel.isnull().sum()

client_id             0
sale_flg              0
sale_amount       17979
contacts              0
feature_1             0
client_segment        0
feature_2             0
feature_3             0
feature_4             0
feature_5             0
feature_6             0
feature_7            11
feature_8            11
feature_9            11
feature_10           11
region_cd            34
dtype: int64

In [7]:
funnel['sale_flg'].value_counts() / len(funnel)

0   0.8363103544515769
1   0.1636896455484231
Name: sale_flg, dtype: float64

In [8]:
funnel['contacts'].value_counts()

1    17361
2     3484
3      577
4       71
5        5
Name: contacts, dtype: int64

In [9]:
funnel['region_cd'].value_counts()

86.0    3131
77.0    1430
54.0    1372
78.0    1223
66.0    1001
        ... 
15.0      11
83.0       5
14.0       4
87.0       1
91.0       1
Name: region_cd, Length: 75, dtype: int64

# Trxn

In [10]:
transactions

,client_id,card_id,tran_time,tran_amt_rur,mcc_cd,merchant_cd,txn_country,txn_city,tsp_name,txn_comment_1,txn_comment_2
0,-1893104556496814867,-6909844117230568380,2019-07-01 00:00:00,6071.0,6011.0,3796217713062573289,RUS,N. NOVGOROD,MOLODEZHNYY PR. DEPO,Cash withdrawal through an ATM,NaN
1,-1893104556496814867,-6909844117230568380,2019-07-29 00:00:00,6071.0,6011.0,3796217713062573289,RUS,N. NOVGOROD,MOLODEZHNYY PR. DEPO,Cash withdrawal through an ATM,NaN
2,-1893104556496814867,-6909844117230568380,2019-08-31 00:00:00,7285.0,6011.0,3796217713062573289,RUS,N. NOVGOROD,MOLODEZHNYY PR. DEPO,Cash withdrawal through an ATM,NaN
3,6886062013213911831,6527755410945490683,2019-08-14 10:38:33,2428.0,6011.0,-774579209084196290,RUS,RADUZHNYI,TTS SITI,Cash withdrawal through an ATM,NaN
4,6886062013213911831,6527755410945490683,2019-07-17 12:27:58,6071.0,6011.0,-7735007003326589238,RUS,RADUZHNYI,ODTS AGANGRAD,Cash withdrawal through an ATM,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3035700,-3405096435133737788,-1484177776108621230,2018-09-06 07:41:17,728.0,6012.0,3967370607447365704,RUS,VISA DIRECT,VISA MONEY TRANSFER,Payment by card (bank transfer),VISA MONEY TRANSFER
3035701,5941620754379828660,-4513708287312715087,2018-12-02 08:16:48,2550.0,6012.0,3967370607447365704,RUS,VISA DIRECT,VISA MONEY TRANSFER,Payment by card (bank transfer),VISA MONEY TRANSFER
3035702,5941620754379828660,-4513708287312715087,2018-12-28 13:55:12,3642.0,6012.0,3967370607447365704,RUS,VISA DIRECT,VISA MONEY TRANSFER,Payment by card (bank transfer),VISA MONEY TRANSFER
3035703,2964514409686635938,4497627322591857070,2018-12-21 09:32:26,2792.0,6012.0,3967370607447365704,RUS,VISA DIRECT,VISA MONEY TRANSFER,Payment by card (bank transfer),VISA MONEY TRANSFER


In [11]:
transactions.isnull().sum()

client_id              0
card_id                0
tran_time              0
tran_amt_rur           0
mcc_cd               132
merchant_cd            0
txn_country            0
txn_city            5385
tsp_name            2162
txn_comment_1          0
txn_comment_2    2638786
dtype: int64

In [12]:
transactions['mcc_cd'] = transactions['mcc_cd'].fillna(-2)
transactions['txn_city'] = transactions['txn_city'].fillna('<UNK>')
transactions['tsp_name'] = transactions['tsp_name'].fillna('<UNK>')
transactions['txn_comment_2'] = transactions['txn_comment_2'].fillna('<UNK>')

In [13]:
transactions['client_id'].value_counts()

 8247514152661289191    3101
-9059132502179716006    2656
 7922361971867610498    2413
-7299536715312723669    2322
 410746725485357352     2318
                        ... 
 9042984261245798495       1
-9164745288538024239       1
-2639946325109418679       1
-3176336415532661554       1
 328258837130732479        1
Name: client_id, Length: 11236, dtype: int64

In [14]:
transactions['tran_time'].value_counts()

2019-02-28 00:00:00    5665
2019-06-02 00:00:00    3819
2019-08-14 00:00:00    3296
2019-08-28 00:00:00    3291
2019-07-24 00:00:00    3270
                       ... 
2019-05-15 13:24:30       1
2019-07-24 10:01:12       1
2019-02-07 04:46:24       1
2019-04-22 18:53:45       1
2019-06-17 10:27:57       1
Name: tran_time, Length: 2082989, dtype: int64

In [15]:
pd.to_datetime(transactions['tran_time']).min(), pd.to_datetime(transactions['tran_time']).max()

(Timestamp('2018-09-01 00:00:00'), Timestamp('2019-08-31 23:58:15'))

In [16]:
data['total_transactions'] = data['client_id'].map(transactions.groupby('client_id').size()).fillna(0)
data['total_transactions'].value_counts()

0.0       10262
2.0         135
11.0        120
12.0        114
1.0         111
          ...  
1060.0        1
1244.0        1
1146.0        1
1346.0        1
1174.0        1
Name: total_transactions, Length: 1258, dtype: int64

In [17]:
data['total_cards'] = get_feature_total(transactions, 'card_id')
data['total_cards'].value_counts()

0.0     10262
1.0      6618
2.0      2818
3.0      1130
4.0       446
5.0       155
6.0        46
7.0        13
8.0         7
9.0         2
13.0        1
Name: total_cards, dtype: int64

In [18]:
data['mean_transaction_amt'] = data['client_id'].map(transactions.groupby('client_id')['tran_amt_rur'].mean()).fillna(-1)
data['mean_transaction_amt'].value_counts()

-1.0                  10262
121.0                    46
1214.0                    8
24282.0                   6
1821.0                    6
                      ...  
10450.125                 1
1471.864406779661         1
1037.6381766381767        1
1337.5238095238096        1
49857.3                   1
Name: mean_transaction_amt, Length: 11121, dtype: int64

In [19]:
data['total_mcc_cd'] = get_feature_total(transactions, 'mcc_cd')
data['total_mcc_cd'].value_counts()

0.0     10262
2.0       683
1.0       620
3.0       546
4.0       318
        ...  
88.0        1
78.0        1
85.0        1
79.0        1
91.0        1
Name: total_mcc_cd, Length: 86, dtype: int64

In [20]:
data['total_share_mcc_cd'] = (data['total_mcc_cd'] / data['total_transactions']).fillna(0)
data['total_share_mcc_cd'].value_counts()

0.0                     10262
1.0                       229
0.5                       162
0.25                      150
0.3333333333333333        140
                        ...  
0.1018867924528302          1
0.051771117166212535        1
0.06791569086651054         1
0.04470588235294118         1
0.0821256038647343          1
Name: total_share_mcc_cd, Length: 4984, dtype: int64

In [21]:
data['most_common_mcc_cd'] = get_feature_most_common(transactions, 'mcc_cd', -1)
data['most_common_mcc_cd'].value_counts()

-1.0      10262
5411.0     5325
6011.0     2595
6536.0      844
5499.0      550
          ...  
5977.0        1
3011.0        1
7273.0        1
8011.0        1
8071.0        1
Name: most_common_mcc_cd, Length: 74, dtype: int64

In [22]:
data['total_merchant_cd'] = get_feature_total(transactions, 'merchant_cd')
data['total_merchant_cd'].value_counts()

0.0      10262
3.0        689
4.0        671
2.0        641
5.0        563
         ...  
199.0        1
115.0        1
385.0        1
317.0        1
209.0        1
Name: total_merchant_cd, Length: 154, dtype: int64

In [23]:
data['total_share_merchant_cd'] = (data['total_merchant_cd'] / data['total_transactions']).fillna(0)
data['total_share_merchant_cd'].value_counts()

0.0                     10262
1.0                       213
0.5                       183
0.3333333333333333        144
0.25                      119
                        ...  
0.040955631399317405        1
0.026410564225690276        1
0.03547297297297297         1
0.08665906499429875         1
0.1446280991735537          1
Name: total_share_merchant_cd, Length: 4942, dtype: int64

In [24]:
data['most_common_merchant_cd'] = get_feature_most_common(transactions, 'merchant_cd', -1)
data['most_common_merchant_cd'].value_counts()

-1.0                      10262
3.9673706074473656e+18     9268
6.303729926858726e+18       346
3.9569210638920535e+18      202
-5.649346468447594e+18      176
                          ...  
-8.964917765656054e+17        1
-4.192612835342741e+18        1
-7.64182099647542e+18         1
-9.16373487481942e+18         1
-6.871520906596311e+18        1
Name: most_common_merchant_cd, Length: 976, dtype: int64

In [25]:
data['total_txn_city'] = get_feature_total(transactions, 'txn_city')
data['total_txn_city'].value_counts()

0.0      10262
2.0       1323
3.0       1003
1.0        983
4.0        788
         ...  
84.0         1
93.0         1
100.0        1
118.0        1
75.0         1
Name: total_txn_city, Length: 92, dtype: int64

In [26]:
data['total_share_txn_city'] = (data['total_txn_city'] / data['total_transactions']).fillna(0)
data['total_share_txn_city'].value_counts()

0.0                     10262
1.0                       165
0.5                       154
0.2                       127
0.25                      124
                        ...  
0.02113821138211382         1
0.02556818181818182         1
0.11009174311926606         1
0.035758323057953144        1
0.06623931623931624         1
Name: total_share_txn_city, Length: 4279, dtype: int64

In [27]:
data['most_common_txn_city'] = get_feature_most_common(transactions, 'txn_city', '<unknown>')
data['most_common_txn_city'].value_counts()

<unknown>        10262
MOSCOW            1819
NOVOSIBIRSK        520
SURGUT             258
KOGALYM            251
                 ...  
TUYMAZY              1
FRYAZINO             1
TRUBCHEVSK           1
KAZACHI LAGER        1
SHUMYACHI            1
Name: most_common_txn_city, Length: 1142, dtype: int64

In [28]:
data['total_tsp_name'] = get_feature_total(transactions, 'tsp_name')
data['total_tsp_name'].value_counts()

0.0      10262
2.0        477
3.0        443
1.0        378
4.0        327
         ...  
263.0        1
238.0        1
226.0        1
298.0        1
293.0        1
Name: total_tsp_name, Length: 302, dtype: int64

In [29]:
data['total_share_tsp_name'] = (data['total_tsp_name'] / data['total_transactions']).fillna(0)
data['total_share_tsp_name'].value_counts()

0.0                    10262
1.0                      263
0.5                      210
0.3333333333333333       169
0.25                     133
                       ...  
0.3522727272727273         1
0.206                      1
0.12867647058823528        1
0.17304492512479203        1
0.27842227378190254        1
Name: total_share_tsp_name, Length: 6030, dtype: int64

In [30]:
data['most_common_tsp_name'] = get_feature_most_common(transactions, 'tsp_name', '<unknown>')
data['most_common_tsp_name'].value_counts()

<unknown>             10262
OPEN.RU CARDCARD       1495
PYATEROCHKA             876
ATM                     700
WWW.OPENONLINE.RU       682
                      ...  
BAR JIGGERPONY            1
IP TADEVOSYAN RS          1
KUZBASSKII BROILER        1
MAGAZIN RADAMIR           1
EST POVOD                 1
Name: most_common_tsp_name, Length: 3477, dtype: int64

In [31]:
data['total_txn_comment_1'] = get_feature_total(transactions, 'txn_comment_1')
data['total_txn_comment_1'].value_counts()

0.0    10262
3.0     2548
4.0     2545
5.0     2161
2.0     2054
6.0     1127
1.0      714
7.0       87
Name: total_txn_comment_1, dtype: int64

In [32]:
data['most_common_txn_comment_1'] = get_feature_most_common(transactions, 'txn_comment_1', '<unknown>')
data['most_common_txn_comment_1'].value_counts()

<unknown>                          10262
Payment for goods and services      8607
Cash withdrawal through an ATM      1333
Payment by card (bank transfer)      480
Cash deposit by card                 440
Cashless transfer                    369
Cash withdrawal                        4
Return of goods / services             3
Name: most_common_txn_comment_1, dtype: int64

In [33]:
data['total_txn_comment_2'] = get_feature_total(transactions, 'txn_comment_2')
data['total_txn_comment_2'].value_counts()

0.0    10262
4.0     3785
3.0     2503
1.0     2142
2.0     2120
5.0      686
Name: total_txn_comment_2, dtype: int64

In [34]:
data['most_common_txn_comment_2'] = get_feature_most_common(transactions, 'txn_comment_2', '<unknown>')
data['most_common_txn_comment_2'].value_counts()

<UNK>                     10428
<unknown>                 10262
<manual mode>               346
Purchase payment (web)      333
Opening Online              111
VISA MONEY TRANSFER          18
Name: most_common_txn_comment_2, dtype: int64